In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
metadata = pd.read_csv('metadata.csv')
metadata = metadata[metadata['time']>=20181201]

mf47 = pd.read_csv('recommend_mf48.csv', header=None)
month3 = pd.read_csv('recommend_3month.csv', header=None)
follow = pd.read_csv('recommend_follow57.csv', header=None)
wr = pd.read_csv('recommend_wr.csv', header=None)

coldstarter = pd.read_csv('cold_starter_id.csv')

In [3]:
def cleaning(rec):
    rec['id'] = rec[0].apply(lambda x: str(x).split(' ')[0])
    rec['recommend'] = rec[0].apply(lambda x: str(x).split(' ')[1:])
    return rec.iloc[:, 1:]

In [4]:
mf47 = cleaning(mf47)
month3 = cleaning(month3)
follow = cleaning(follow)
wr = cleaning(wr)

In [ ]:
%%time
mf47['submit']=''
for idx in range(len(mf47)):
    
    a = month3.loc[idx, 'recommend']
    b = mf47.loc[idx, 'recommend'][:20]
    c = follow.loc[idx, 'recommend']
    d = wr.loc[idx, 'recommend'][:20]

    recommend_list = []
    recommend_list.extend([a[0]])
    a = a[1:]

    while True:
        try:
            recommend_list.extend(c[:2])
            c = c[2:]
            recommend_list.extend(b[:2])
            b = b[2:]
        except:
            pass
        if len(c)+len(b)==0:
            break
            
    recommend_list = pd.Series(recommend_list).unique().tolist()
    
    recommend_list3 = []
    while True:
        try:
            recommend_list3.extend(recommend_list[:2])
            recommend_list = recommend_list[2:]
            recommend_list3.extend(d[:2])
            d = d[2:]
        except:
            pass
        if len(recommend_list)+len(d)==0:
            break
            
    recommend_list = pd.Series(recommend_list3).unique().tolist()

    recommend_list2 = []
    while True:
        try:
            recommend_list2.extend(recommend_list[:2])
            recommend_list = recommend_list[2:]
            recommend_list2.extend(a[:2])
            a = a[2:]
        except:
            pass
        if len(recommend_list2)>=100:
            break
    
    recommend_list2 = pd.Series(recommend_list2).unique().tolist()[:80]
    mf47.loc[idx, 'submit'] = recommend_list2

In [ ]:
rec_list = np.unique([j for i in mf47['submit'] for j in i])
rec_list_market = metadata['id'][~metadata['id'].isin(rec_list)].unique()

In [ ]:
random.seed(42)
random.shuffle(rec_list_market)
for idx in range(len(mf47)):
    recommend_list = mf47.loc[idx, 'submit']
    recommend_list.extend(rec_list_market[:20].tolist())
    rec_list_market = rec_list_market[20:]
    mf47.loc[idx, 'submit'] = recommend_list

In [ ]:
dev = mf47
dev = dev.drop(columns=['recommend']).rename(columns={'submit':'recommend'})
dev['submit'] = dev['id'] + ' ' + dev['recommend'].apply(lambda x: ' '.join(x))
dev['submit'].to_csv('recommend.csv', index=False)
print('100개가 완벽히 추천된 아이템 개수 : {} \n추천된 item의 unique개수 : {} - entropy와 밀접한 관련'.format(
    sum(dev['submit'].apply(lambda x: len(x.split(' ')))==101), len(np.unique([j for i in dev['recommend'] for j in i]))))